In [1]:
import tensorflow as tf
import inception_v3
import vgg
from datagenerator import ImageDataGenerator
from tensorflow.contrib.data import Iterator
import numpy as np
from datetime import datetime
import time
import os
import json
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Use same number for training and validation
# Ex) 0th folding -> 'train0' for training, 'valid0' for validation
data_txt = open('/hrlab-sf/icros/data_split/5fold_0802.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)
print(tr_data_dir.keys())

dict_keys(['valid3', 'train3', 'train4', 'valid4', 'valid1', 'valid0', 'train2', 'train1', 'valid2', 'train0'])


In [3]:
# Path to the textfiles for the trainings and validation set
num = 0
date = "20180810"
train_file = './train%d_HR_ICROS.txt'%num
val_file1 = './valid%d_HR_ICROS.txt'%num
batch_size = 32
num_classes = 2
learning_rate = 0.0001
training_epochs = 30
display_step = 20

# Path for tf.summary.FileWriter and to store model checkpoints
filewriter_path = "/hdd3/dhj_container/ICROS_vgg/vgg0811%d/"% num
checkpoint_path = "/hdd3/dhj_container/ICROS_vgg/vggch0811%d"% num

In [4]:
with tf.device('/gpu:0'):
    # data load
    tr_data = ImageDataGenerator(train_file,
                                 mode='training',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=False)
    val_data1 = ImageDataGenerator(val_file1,
                                  mode='inference',
                                  batch_size=batch_size,
                                  num_classes=num_classes,
                                  shuffle=False)

    # create an reinitializable iterator given the dataset structure
    iterator = Iterator.from_structure(tr_data.data.output_types,
                                           tr_data.data.output_shapes)

    next_batch = iterator.get_next()
    

    # Ops for initializing the two different iterators
    training_init_op = iterator.make_initializer(tr_data.data)
    validation_init_op1 = iterator.make_initializer(val_data1.data)

    # TF placeholder for graph input and output
    x = tf.placeholder(tf.float32, [batch_size, 224, 224, 3])
    y = tf.placeholder(tf.float32, [batch_size, num_classes])



    net, net_points = vgg.vgg_16(x,
                                num_classes=num_classes,
                                dropout_keep_prob=0.5
                               )
        
    # Op for calculating the loss
    with tf.name_scope("cross_ent"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=y))
                
    # Train op
    with tf.name_scope("train"):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=y))
        train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
            
    # Add the loss to summary
    tf.summary.scalar('cross_entropy', loss)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Add the accuracy to the summary
    tf.summary.scalar('accuracy', accuracy)
    
    # Merge all summaries together
    merged_summary = tf.summary.merge_all()

    # Initialize the FileWriter
    writer = tf.summary.FileWriter(filewriter_path)

    # Initialize an saver for store model checkpoints
    saver = tf.train.Saver()

    train_batches_per_epoch = int(np.floor(tr_data.data_size / batch_size))
    val_batches_per_epoch1 = int(np.floor(val_data1.data_size / batch_size)) 
    print(train_batches_per_epoch)
    print(val_batches_per_epoch1)

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.
114
28


In [5]:
# Training
vaild_result = np.array([])
vaild_result.resize((training_epochs,1))


config=tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
   
    print("{} Start training...".format(datetime.now()))
    print("{} Open tensorboard --logdir={}".format(datetime.now(),
                                                      filewriter_path))
    
    img_batch = np.zeros((batch_size,224,224,3), dtype ='uint8')
    
    for epoch in range(training_epochs):
       
        print("{} Epoch number: {}".format(datetime.now(), epoch+1))
       
        # Initialize iterator with the training dataset
        sess.run(training_init_op)

        for step in range(train_batches_per_epoch):

            # get next batch of data
            img_batch, label_batch = sess.run(next_batch)       

            # And run the training op
            sess.run(train_op, feed_dict={x: img_batch, y: label_batch})

            # Generate summary with the current batch of data and write to file
            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: img_batch,
                                                        y: label_batch})
                writer.add_summary(s, epoch*train_batches_per_epoch + step)
                print("{} {} step".format(datetime.now(), step))

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        sess.run(validation_init_op1)
        test_acc = 0.
        test_count = 0
        wrong_cnt = 0
        pre = np.array([])
        pre.resize((val_batches_per_epoch1,batch_size))
        
        for a in range(val_batches_per_epoch1):

            img_batch, label_batch = sess.run(next_batch)
            acc = sess.run(accuracy, feed_dict={x: img_batch,
                                                y: label_batch})
            pre[test_count] = sess.run(correct_prediction, feed_dict={x: img_batch,
                                                y: label_batch})
            
            if epoch == (training_epochs-1):
                pre = pre.astype('uint32')
                for i in range(batch_size):
                    if pre[test_count][i] == False:
                        order = step*batch_size + i
                        set_num = num+1
                        temp = tr_data_dir['train%d'%set_num][order]
                        name = temp.split('/')[-1].split('.')[0].split('(')[1].split(')')[0]
                        print(name)
            
            test_acc += acc
            test_count += 1
            
        test_acc /= test_count
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                       test_acc))
        vaild_result[epoch] = test_acc
        
        if epoch == training_epochs-1 :
            print("{} Saving checkpoint of model...".format(datetime.now()))
            # save checkpoint of the model
            checkpoint_name = os.path.join(checkpoint_path,
                                           'model_epoch'+str(epoch+1)+'.ckpt')
            save_path = saver.save(sess, checkpoint_name)

            print("{} Model checkpoint saved at {}".format(datetime.now(),
                                                           checkpoint_name))

2018-08-11 07:52:24.617092 Start training...
2018-08-11 07:52:24.624662 Open tensorboard --logdir=/hdd3/dhj_container/ICROS_vgg/vgg08110/
2018-08-11 07:52:24.626385 Epoch number: 1
2018-08-11 07:52:29.000325 0 step
2018-08-11 07:52:38.869721 20 step
2018-08-11 07:52:51.565032 40 step
2018-08-11 07:53:03.623183 60 step
2018-08-11 07:53:15.420753 80 step
2018-08-11 07:53:27.479306 100 step
2018-08-11 07:53:35.079703 Start validation
2018-08-11 07:53:53.477166 Validation Accuracy = 0.7467
2018-08-11 07:53:53.477297 Epoch number: 2
2018-08-11 07:53:54.036449 0 step
2018-08-11 07:54:00.494060 20 step
2018-08-11 07:54:07.010925 40 step
2018-08-11 07:54:13.559232 60 step
2018-08-11 07:54:20.116935 80 step
2018-08-11 07:54:26.686288 100 step
2018-08-11 07:54:30.900354 Start validation
2018-08-11 07:54:36.708788 Validation Accuracy = 0.7712
2018-08-11 07:54:36.709363 Epoch number: 3
2018-08-11 07:54:37.190961 0 step
2018-08-11 07:54:43.723565 20 step
2018-08-11 07:54:50.311445 40 step
2018-08-1

2018-08-11 08:10:02.579596 Validation Accuracy = 0.9844
2018-08-11 08:10:02.579789 Epoch number: 24
2018-08-11 08:10:03.065239 0 step
2018-08-11 08:10:09.680468 20 step
2018-08-11 08:10:16.271877 40 step
2018-08-11 08:10:22.980107 60 step
2018-08-11 08:10:29.672926 80 step
2018-08-11 08:10:36.253441 100 step
2018-08-11 08:10:40.496105 Start validation
2018-08-11 08:10:46.487713 Validation Accuracy = 0.9866
2018-08-11 08:10:46.487818 Epoch number: 25
2018-08-11 08:10:46.991632 0 step
2018-08-11 08:10:53.566389 20 step
2018-08-11 08:11:00.156340 40 step
2018-08-11 08:11:06.852939 60 step
2018-08-11 08:11:13.447923 80 step
2018-08-11 08:11:20.073119 100 step
2018-08-11 08:11:24.320888 Start validation
2018-08-11 08:11:30.409641 Validation Accuracy = 0.9888
2018-08-11 08:11:30.409765 Epoch number: 26
2018-08-11 08:11:30.893437 0 step
2018-08-11 08:11:37.479466 20 step
2018-08-11 08:11:44.024237 40 step
2018-08-11 08:11:50.653225 60 step
2018-08-11 08:11:57.258265 80 step
2018-08-11 08:12:0

In [6]:
vaild_result

array([[0.74665179],
       [0.77120536],
       [0.85714286],
       [0.91294643],
       [0.91741071],
       [0.92299107],
       [0.91071429],
       [0.93973214],
       [0.94642857],
       [0.97098214],
       [0.95982143],
       [0.95758929],
       [0.95647321],
       [0.96540179],
       [0.97433036],
       [0.97433036],
       [0.96651786],
       [0.96875   ],
       [0.953125  ],
       [0.97544643],
       [0.96763393],
       [0.984375  ],
       [0.984375  ],
       [0.98660714],
       [0.98883929],
       [0.984375  ],
       [0.98660714],
       [0.984375  ],
       [0.98772321],
       [0.98549107]])

# 맞추면 1, 틀리면 0
test_count = 0
config=tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
config.gpu_options.allow_growth = True
pre = np.array([])
pre.resize((val_batches_per_epoch1,batch_size))
with tf.Session(config=config, graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer())
    img_batch = np.zeros((batch_size,227,227,3), dtype ='uint8')
    for epoch in range(training_epochs):
        # Initialize iterator with the training dataset
        sess.run(validation_init_op1)
        for step in range(val_batches_per_epoch1):
            # get next batch of data
            img_batch, label_batch = sess.run(next_batch)    
            #print(len(label_batch))
            pre[test_count] = sess.run(correct_prediction, feed_dict={x: img_batch,
                                                y: label_batch})
            pre = pre.astype('uint32')
            for i in range(batch_size):
                if pre[test_count][i] == False:
                    order = step*batch_size + i
                    set_num = num+1
                    temp = tr_data_dir['train%d'%set_num][order]
                    name = temp.split('/')[-1].split('.')[0].split('(')[1].split(')')[0]
                    print(name)
            test_count+=1
            
